### Imports

In [42]:
import json
import os

import pandas as pd

from utils.utils_go import *

# %load_ext autotime

### Parameters

In [43]:
params = {
    "methods": ["vgae-base"], # ["vgae", "dgi", "dgi-tran", "dgi-indu", "argva-base", "vgae-line", "vgae-base"], # ["vgae", "dgi", "dgi-tran", "dgi-indu", "argva-base", "vgae-line", "vgae-base"]
    "data_variations": ["str"], # ["none", "str", "dyn"]
    "has_transformation": False, # True or False # change
    "control": "HN", # "T" # "LG" # "FCSglc" # "WT"
    "dimension": 3,
    "threshold_corr": 0.01,
    "threshold_log2": 0,
    "alpha": 0.05,
    "iterations": 2,
    "raw_data_file": "leaves_format", # "reinhard" # "mutant_a" # "plant" # "single_cell" # ""Trial Alfredo process" # "Trial 1_Reinhard" # "Trial Alfredo" # change
    "obs": "",
    "seeds": [42, 43, 44, 45, 46]
}

In [44]:
dir_path = "output"

res = sorted(os.listdir(dir_path))
n = len(res)
exp = "exp{}".format(n)
exp

'exp11'

### Load dataset

In [45]:
# load dataset groups
df_raw = pd.read_csv("raw_data/{}.csv".format(params["raw_data_file"]), delimiter="|")
# df_raw = pd.read_excel("raw_data/{}.xlsx".format(params["raw_data_file"]))
df_raw

,Alignment ID,Average Mz,Metabolite name,HN_1.1,HN_1.2,HN_1.3,HN_2.1,HN_2.2,HN_2.3,HV_1.1,HV_1.2,HV_1.3,HV_2.1,HV_2.3,HV_2.4,HV_2.5
0,0,58.06574,NaN,173.67982,1.17128,1.17128,1.17128,1.17128,117.12768,1.17128,1.17128,1.17128,1.17128,1.17128,1.17128,1.17128
1,1,65.26001,NaN,241.33417,2.41334,2.41334,2.41334,2.41334,306.20944,2.41334,2.41334,2.41334,2.41334,2.41334,2.41334,2.41334
2,2,65.26541,NaN,3.46233,3.46233,3.46233,3.46233,3.46233,346.23291,3.46233,3.46233,3.46233,3.46233,3.46233,3.46233,3.46233
3,3,65.38704,NaN,117.64917,1.17649,1.17649,1.17649,1.17649,132.44109,1.17649,1.17649,1.17649,1.17649,1.17649,1.17649,1.17649
4,4,65.87144,NaN,144.98218,1.44982,1.44982,230.19943,1.44982,1.44982,1.44982,1.44982,1.44982,1.44982,1.44982,1.44982,1.44982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1857,1857,717.61292,NaN,1.96126,1.96126,1.96126,1.96126,1.96126,1.96126,1.96126,1.96126,1.96126,196.12572,1.96126,1.96126,1.96126
1858,1858,723.19322,NaN,1.00339,1.00339,1.00339,346.97250,100.33888,183.92720,1.00339,1.00339,1.00339,1.00339,1.00339,1.00339,1.00339
1859,1859,723.95593,NaN,1.39055,1.39055,1.39055,1.39055,139.05484,1.39055,1.39055,1.39055,1.39055,1.39055,1.39055,1.39055,1.39055
1860,1860,723.95758,NaN,1.77838,606.15350,177.83754,703.60211,1.77838,1.77838,1.77838,1.77838,1.77838,1.77838,1.77838,1.77838,1.77838


### Format dataset

In [46]:
# has transformation
columns_data = list(df_raw.columns)[3:]
if params["has_transformation"]:
    print("transformation")
    for column in columns_data:
        df_raw[column] = df_raw[column].apply(lambda x: 10**x)
df_raw

,Alignment ID,Average Mz,Metabolite name,HN_1.1,HN_1.2,HN_1.3,HN_2.1,HN_2.2,HN_2.3,HV_1.1,HV_1.2,HV_1.3,HV_2.1,HV_2.3,HV_2.4,HV_2.5
0,0,58.06574,NaN,173.67982,1.17128,1.17128,1.17128,1.17128,117.12768,1.17128,1.17128,1.17128,1.17128,1.17128,1.17128,1.17128
1,1,65.26001,NaN,241.33417,2.41334,2.41334,2.41334,2.41334,306.20944,2.41334,2.41334,2.41334,2.41334,2.41334,2.41334,2.41334
2,2,65.26541,NaN,3.46233,3.46233,3.46233,3.46233,3.46233,346.23291,3.46233,3.46233,3.46233,3.46233,3.46233,3.46233,3.46233
3,3,65.38704,NaN,117.64917,1.17649,1.17649,1.17649,1.17649,132.44109,1.17649,1.17649,1.17649,1.17649,1.17649,1.17649,1.17649
4,4,65.87144,NaN,144.98218,1.44982,1.44982,230.19943,1.44982,1.44982,1.44982,1.44982,1.44982,1.44982,1.44982,1.44982,1.44982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1857,1857,717.61292,NaN,1.96126,1.96126,1.96126,1.96126,1.96126,1.96126,1.96126,1.96126,1.96126,196.12572,1.96126,1.96126,1.96126
1858,1858,723.19322,NaN,1.00339,1.00339,1.00339,346.97250,100.33888,183.92720,1.00339,1.00339,1.00339,1.00339,1.00339,1.00339,1.00339
1859,1859,723.95593,NaN,1.39055,1.39055,1.39055,1.39055,139.05484,1.39055,1.39055,1.39055,1.39055,1.39055,1.39055,1.39055,1.39055
1860,1860,723.95758,NaN,1.77838,606.15350,177.83754,703.60211,1.77838,1.77838,1.77838,1.77838,1.77838,1.77838,1.77838,1.77838,1.77838


In [47]:
# concat
df_join_raw = pd.concat([
    df_raw.iloc[:, :]], axis=1)
df_join_raw.set_index("Alignment ID", inplace=True)
df_join_raw

,Average Mz,Metabolite name,HN_1.1,HN_1.2,HN_1.3,HN_2.1,HN_2.2,HN_2.3,HV_1.1,HV_1.2,HV_1.3,HV_2.1,HV_2.3,HV_2.4,HV_2.5
Alignment ID,,,,,,,,,,,,,,,
0,58.06574,NaN,173.67982,1.17128,1.17128,1.17128,1.17128,117.12768,1.17128,1.17128,1.17128,1.17128,1.17128,1.17128,1.17128
1,65.26001,NaN,241.33417,2.41334,2.41334,2.41334,2.41334,306.20944,2.41334,2.41334,2.41334,2.41334,2.41334,2.41334,2.41334
2,65.26541,NaN,3.46233,3.46233,3.46233,3.46233,3.46233,346.23291,3.46233,3.46233,3.46233,3.46233,3.46233,3.46233,3.46233
3,65.38704,NaN,117.64917,1.17649,1.17649,1.17649,1.17649,132.44109,1.17649,1.17649,1.17649,1.17649,1.17649,1.17649,1.17649
4,65.87144,NaN,144.98218,1.44982,1.44982,230.19943,1.44982,1.44982,1.44982,1.44982,1.44982,1.44982,1.44982,1.44982,1.44982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1857,717.61292,NaN,1.96126,1.96126,1.96126,1.96126,1.96126,1.96126,1.96126,1.96126,1.96126,196.12572,1.96126,1.96126,1.96126
1858,723.19322,NaN,1.00339,1.00339,1.00339,346.97250,100.33888,183.92720,1.00339,1.00339,1.00339,1.00339,1.00339,1.00339,1.00339
1859,723.95593,NaN,1.39055,1.39055,1.39055,1.39055,139.05484,1.39055,1.39055,1.39055,1.39055,1.39055,1.39055,1.39055,1.39055


In [48]:
# split
df_join_raw = df_join_raw.rename_axis(None)
# df_join_raw = df_join_raw.iloc[:, 2:]
df_join_raw

,Average Mz,Metabolite name,HN_1.1,HN_1.2,HN_1.3,HN_2.1,HN_2.2,HN_2.3,HV_1.1,HV_1.2,HV_1.3,HV_2.1,HV_2.3,HV_2.4,HV_2.5
0,58.06574,NaN,173.67982,1.17128,1.17128,1.17128,1.17128,117.12768,1.17128,1.17128,1.17128,1.17128,1.17128,1.17128,1.17128
1,65.26001,NaN,241.33417,2.41334,2.41334,2.41334,2.41334,306.20944,2.41334,2.41334,2.41334,2.41334,2.41334,2.41334,2.41334
2,65.26541,NaN,3.46233,3.46233,3.46233,3.46233,3.46233,346.23291,3.46233,3.46233,3.46233,3.46233,3.46233,3.46233,3.46233
3,65.38704,NaN,117.64917,1.17649,1.17649,1.17649,1.17649,132.44109,1.17649,1.17649,1.17649,1.17649,1.17649,1.17649,1.17649
4,65.87144,NaN,144.98218,1.44982,1.44982,230.19943,1.44982,1.44982,1.44982,1.44982,1.44982,1.44982,1.44982,1.44982,1.44982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1857,717.61292,NaN,1.96126,1.96126,1.96126,1.96126,1.96126,1.96126,1.96126,1.96126,1.96126,196.12572,1.96126,1.96126,1.96126
1858,723.19322,NaN,1.00339,1.00339,1.00339,346.97250,100.33888,183.92720,1.00339,1.00339,1.00339,1.00339,1.00339,1.00339,1.00339
1859,723.95593,NaN,1.39055,1.39055,1.39055,1.39055,139.05484,1.39055,1.39055,1.39055,1.39055,1.39055,1.39055,1.39055,1.39055
1860,723.95758,NaN,1.77838,606.15350,177.83754,703.60211,1.77838,1.77838,1.77838,1.77838,1.77838,1.77838,1.77838,1.77838,1.77838


In [49]:
# get groups name
groups_id = []
for item in df_join_raw.iloc[:, 2:].columns.values:
    group_id = item.split("_")[0]
    if group_id not in groups_id:
        groups_id.append(group_id)
groups_id

['HN', 'HV']

In [50]:
# get subgroups names
subgroups_id = get_subgroups_id(df_join_raw, groups_id)
subgroups_id

{'HN': ['1', '2'], 'HV': ['1', '2']}

In [51]:
# get groups combination
groups = []
control = params["control"]

groups = []
properties = list(subgroups_id.keys())
properties.remove(control)
for group in properties:
    aux = [control, group]
    groups.append(aux)

### Create folders

In [52]:
# create experiments folder
try: 
    os.mkdir("output/{}".format(exp))
    os.mkdir("output/{}/correlations".format(exp))
    os.mkdir("output/{}/preprocessing".format(exp))
    os.mkdir("output/{}/preprocessing/edges".format(exp))
    os.mkdir("output/{}/preprocessing/graphs".format(exp))
    os.mkdir("output/{}/preprocessing/graphs_data".format(exp))
    os.mkdir("output/{}/node_embeddings".format(exp))
    os.mkdir("output/{}/edge_embeddings".format(exp))
    os.mkdir("output/{}/common_edges".format(exp))
    os.mkdir("output/{}/changes".format(exp))
    os.mkdir("output/{}/plots".format(exp))
    os.mkdir("output/{}/biocyc".format(exp))
except OSError as error: 
    print(error)

### Save dataset and parameters

In [53]:
# save dataset
df_join_raw.to_csv("input/{}_raw.csv".format(exp), index=True)

# save parameters
parameters = {
    "exp": exp,
    "methods": params["methods"],
    "data_variations": params["data_variations"],
    "has_transformation": params["has_transformation"],
    "control": params["control"],
    "dimension": params["dimension"],
    "threshold_corr": params["threshold_corr"],
    "threshold_log2": params["threshold_log2"],
    "alpha": params["alpha"],
    "iterations": params["iterations"],
    
    "groups_id": groups_id,
    "subgroups_id": subgroups_id,
    "groups": groups,

    "seeds": params["seeds"],
    "obs": params["obs"]
}

with open("output/{}/parameters.json".format(exp), "w") as outfile:
    json.dump(parameters, outfile, indent=4)

In [54]:
experiments = {
    "exp": exp
}

with open("exp.json".format(experiments), "w") as outfile:
    json.dump(experiments, outfile, indent=4)